# 🧮 Applied Generalized Eigenproblem (GEP) Examples and Bridge to GSVD
*(Notebook 3 in the Generalized Eigenproblem → GSVD arc)*

## 0  Setup & Goals

**Purpose**  
We extend the generalized eigenproblem
$$A x = \lambda B x$$  
into applied and computational contexts, exploring how eigenstructure evolves across engineering and data-analytic systems, and how these ideas lead naturally to the **generalized SVD**.

**Objectives**
- Connect physical and data examples of GEP.
- Visualize modes and eigenvalue motion interactively.
- Examine conditioning effects.
- Motivate the GSVD.

In [ ]:
import panel as pn, holoviews as hv, numpy as np
pn.extension('katex','plotly')
hv.extension('bokeh')

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

## 1  Revisiting the GEP Framework

In [ ]:
pn.pane.LaTeX(r"""\\displaystyle A x = \\lambda B x,\\qquad
\\text{with } A,B\\in\\mathbb{R}^{n\\times n},\\;
x\\neq 0""", styles={'font-size':'20pt'})

> **Reminder:**  
> When $B$ is positive definite, the problem defines a *weighted inner product*  
> $ \langle x,y\rangle_B = x^T B y $.  
> Eigenvectors for distinct eigenvalues are $B$-orthogonal.

## 2  Case Study 1 — Three-Mass Spring System

In [ ]:
Main.eval('''
using LinearAlgebra
function KM_matrices(masses, springs)
    n = length(masses)
    K = zeros(Float64,n,n)
    for i in 1:n
        if i>1
            K[i,i]   += springs[i]
            K[i,i-1] -= springs[i]
            K[i-1,i] -= springs[i]
            K[i-1,i-1] += springs[i]
        end
    end
    M = Diagonal(masses)
    return K,M
end
''')

In [ ]:
m1 = pn.widgets.FloatSlider(name='Mass 1',start=1,end=5,step=0.5,value=2)
m2 = pn.widgets.FloatSlider(name='Mass 2',start=1,end=5,step=0.5,value=2)
m3 = pn.widgets.FloatSlider(name='Mass 3',start=1,end=5,step=0.5,value=2)
k1 = pn.widgets.FloatSlider(name='Spring 1',start=10,end=100,step=5,value=50)
k2 = pn.widgets.FloatSlider(name='Spring 2',start=10,end=100,step=5,value=60)
k3 = pn.widgets.FloatSlider(name='Spring 3',start=10,end=100,step=5,value=40)

In [ ]:
def compute_modes(m1v,m2v,m3v,k1v,k2v,k3v):
    Main.masses  = [m1v,m2v,m3v]
    Main.springs = [k1v,k2v,k3v]
    Main.eval('K,M = KM_matrices(masses,springs)')
    eig = Main.eval('eigen(K,M)')
    lam,vec = np.array(eig.values), np.array(eig.vectors)
    freqs = np.sqrt(np.maximum(lam,0))
    return freqs,vec

def plot_modes(m1v,m2v,m3v,k1v,k2v,k3v):
    freqs,vec = compute_modes(m1v,m2v,m3v,k1v,k2v,k3v)
    curves=[]; x=np.arange(1,4)
    for i,f in enumerate(freqs):
        curves.append(hv.Curve((x,vec[:,i]),label=f'Mode {i+1}: ω={f:.2f}'))
    return hv.Overlay(curves).opts(legend_position='right',xlabel='Mass Index',ylabel='Mode Shape')

interactive_modes = pn.bind(plot_modes,m1,m2,m3,k1,k2,k3)
pn.Column(pn.pane.Markdown('### Mode Shapes for a 3-Mass Spring System'),
          pn.Row(pn.Column(m1,m2,m3),pn.Column(k1,k2,k3)),hv.DynamicMap(interactive_modes))

## 3  Case Study 2 — RLC Circuit Analogue

In [ ]:
def circuit_example(Lvals,Cvals):
    Main.L = np.diag(Lvals)
    Main.C = np.diag(Cvals)
    eig = Main.eval('eigen(inv(L),C)')
    lam = np.array(eig.values)
    return np.sqrt(np.maximum(lam,0))

freqs = circuit_example([1e-3,2e-3,1.5e-3],[1e-6,1.2e-6,0.8e-6])
hv.Curve((np.arange(1,4),freqs)).opts(title='RLC Resonant Frequencies',ylabel='ω (rad/s)')

## 4  Case Study 3 — Data-Analytic GEP and the Path to GSVD

In [ ]:
rng=np.random.default_rng(1)
X=rng.normal(size=(100,3))
Y=X@np.diag([0.9,0.6,0.3])+0.1*rng.normal(size=(100,3))

Main.X,Main.Y = X,Y
Main.eval('''A = X' * Y; B = X' * X; C = Y' * Y; E = eigen(inv(B)*A*inv(C)*A')''')
vals = np.sqrt(np.maximum(np.array(Main.eval('E.values')),0))
hv.Bars((np.arange(1,4),vals)).opts(title='Canonical Correlations (√λ from GEP)',ylabel='Correlation')

## 5  Numerical Conditioning Demo

In [ ]:
def sensitivity_demo(eps=1e-3):
    A=np.array([[3,1],[1,3]],float)
    B=np.array([[2,0],[0,1+eps]],float)
    Main.A,Main.B=A,B
    lam=np.array(Main.eval('eigen(A,B).values'))
    return lam

eps_vals=np.logspace(-3,-1,30)
lam_stack=np.array([sensitivity_demo(e) for e in eps_vals])
hv.Curve((eps_vals,lam_stack[:,0]),label='λ₁') * hv.Curve((eps_vals,lam_stack[:,1]),label='λ₂')

## 6  Summary and Bridge to GSVD

In [ ]:
pn.pane.Markdown('| Concept | Equation | Key Idea |\n|:--|:--|:--|\n| Standard EP | $A x = \lambda x$ | Unweighted modes |\n| Generalized EP | $A x = \lambda B x$ | Two metrics, $B$-orthogonality |\n| GSVD | $A=U Σ_A X^T, B=V Σ_B X^T$ | Joint basis for paired subspaces |')

## 7  Interactive Dashboard (Preview)

In [ ]:
selector = pn.widgets.Select(name='Example',options=['Mass-Spring','Circuit','Data'])
def show_example(sel):
    if sel=='Mass-Spring': return interactive_modes
    if sel=='Circuit': return hv.Curve((np.arange(1,4),freqs))
    if sel=='Data': return hv.Bars((np.arange(1,4),vals))
dashboard = pn.Column('# GEP Example Explorer',selector,pn.bind(show_example,selector))
dashboard.servable()

## 8  Closing Reflection

- **Unifying view:** from vibrating masses to correlated datasets, the same algebraic structure governs modal relationships.  
- **Numerical perspective:** well-conditioned B ensures stable eigenpairs.  
- **Looking ahead:** the **GSVD** extends these ideas to non-square and rank-deficient pairs.